[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ZachDorm/basal_and_bark/blob/main/docs/examples/fia_db.ipynb)

In [ ]:
%pip install basal_and_bark

In [1]:
import basal_and_bark
from basal_and_bark import basal_and_bark as basal
import ipyleaflet
from ipyleaflet import WidgetControl
import ipywidgets
from ipywidgets import HTML, widgets

Set up the Google Earth Engine (GEE) session. If you do not have a Google Earth Engine account, you can create one and authenticate it in the following code block. If you do not wish to make a GEE account, comment out the next block, second to last line of code: print(leaflet_map.addGEEData(county))

There are additional functionalities to this package outside of what can be done using GEE, although it is cooler with GEE.


In [2]:
import ee
#ee.Authenticate()

Next, setup a simple basal_and_bark Map() environment to show functionality of the package in.

In [3]:
leaflet_map = basal.Map(center = [40,-100], zoom = 4, test = "test", scroll_wheel_zoom = True)

       ESTIMATE
0  1.804803e+07
1  6.093884e+05
2  2.706504e+04
3  4.040822e+04
4  1.223426e+07
5  1.606379e+05
6  3.324307e+06
The above table are estimates for 6 different categories of land, as specified by USFS: 'Timberland', 'Reserved', 'Other Forestland', 'Nonforest', 'Non-Census Water', 'Census Water', 'Other'
The table below contains summary statistics of the canopy cover in the Hansen et al Forest Change dataset.
       GEEMap Data
max   1.000000e+02
mean  5.254678e+01
min   0.000000e+00
std   4.517764e+01
sum   1.002224e+10


This is not a state in the USFS Southern Research Station.
The above table are estimates for 6 different categories of land, as specified by USFS: 'Timberland', 'Reserved', 'Other Forestland', 'Nonforest', 'Non-Census Water', 'Census Water', 'Other'
The table below contains summary statistics of the canopy cover in the Hansen et al Forest Change dataset.
       GEEMap Data
max   1.000000e+02
mean  1.880011e+00
min   0.000000e+00
std   1.057519e+01
sum   5.844621e+08


In [4]:
leaflet_map.add_layer_control()

In [5]:
leaflet_map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

The addRefData() function loads in a shp file of the states for data aggregation. This is specific to states currently. Future work will be looking into potential uses of custom scales for examining forestry data in this package.

In [6]:
states = leaflet_map.addRefData()


Setup an output widget that will house a dropdown menu of years, from which the user can select the year they want to get FIA estimates for.

In [7]:
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_widget.clear_output()
ctrl = WidgetControl(widget=output_widget, position='bottomright')
leaflet_map.add_control(ctrl)

dropdown = widgets.Dropdown(
            options = ["2018", "2019", "2020", "2021"], 
            value="2020",
            description='Year',
            )

with output_widget:
            display(dropdown)

This function activates when the user clicks a location on the map - *on_interaction(handle_click)*. A point is added to the map at the location of the click *makePointsFromClick()*. This location data is then used to determine which state the user clicked within *findInt()*. Then FIA estimates are returned for the state the user clicked in *getAPIData()*. Additionally, the Hansen et al forestry data available on GEE is clipped to the state the user clicked in and plotted on the map *addGEEData()*. Summary statistics of this clipped data is also returned. This gives the user two different representations of the state of forests in the state they clicked in.

In [8]:
def handle_click(**kwargs):

    if kwargs.get('type') == 'click':
        output_widget.clear_output()
        #leaflet_map.add_layer(Marker(location=kwargs.get('coordinates')))

        gdf = leaflet_map.makePointsFromClick(kwargs.get('coordinates'))

        state = leaflet_map.findInt(states, gdf)
        
        with output_widget:
            display(dropdown)
            print(state)
        print(leaflet_map.getAPIdata(state, dropdown.value))#['ESTIMATE'])
        print("The above table are estimates for 6 different categories of land, as specified by USFS: "+ 
              "'Timberland', 'Reserved', 'Other Forestland', 'Nonforest', 'Non-Census Water', 'Census Water', 'Other'")
        print("The table below contains summary statistics of the canopy cover in the Hansen et al Forest Change dataset.")
        print(leaflet_map.addGEEData(state))

leaflet_map.on_interaction(handle_click)